In [20]:
from kiteconnect import KiteConnect
import pandas as pd
from selenium import webdriver
import time
import os
import datetime as dt
pd.options.mode.chained_assignment = None

In [2]:
token_path = 'api_key.txt'
key_secret = open(token_path,'r').read().split()   #gives a list object
kite = KiteConnect(api_key = key_secret[0])
#virtually start the chromedriver -  don't make the window pop up
service = webdriver.chrome.service.Service('C:/Users/crypt/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe')
service.start()
#don't render the browser, save some RAM
options = webdriver.ChromeOptions()
options.add_argument('--headless')
#just aggregating all the capabilities like headless, memoryless etc
options = options.to_capabilities()
#start the driver, pass the options 
driver = webdriver.Remote(service.service_url,options)
driver.get(kite.login_url())
driver.implicitly_wait(2)
#fint the element in the webpage by the HTML path
username = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
password = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
username.send_keys(key_secret[2])
password.send_keys(key_secret[3])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
pin = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input')
pin.send_keys(key_secret[4])
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[3]/button').click()
time.sleep(2)
request_token = driver.current_url.split('=')[1].split('&action')[0]
with open('request_token.txt','w') as file:
    file.write(request_token)
driver.quit()

#Use request_token and get_access_token
request_token = open('request_token.txt','r').read().split()
key_secret = open('api_key.txt','r').read().split()
kite = KiteConnect(api_key = key_secret[0])
data = kite.generate_session(request_token[0],api_secret = key_secret[1])
with open('access_token.txt','w') as file:
    file.write(data['access_token'])
    
# Set access token and you're all set!!!!!!!!!!   
kite.set_access_token(open('access_token.txt','r').read().split()[0])

In [4]:
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)

In [5]:
def instrumentLookup(symbol):
    global instrument_df
    try: 
        return instrument_df[instrument_df.tradingsymbol == symbol].instrument_token.values[0]
    except:
        return -1

In [6]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    # Fetch 5 minute OHLC for the last 4 days 
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [9]:
def atr(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['high']-df['low']) # return a positive value for this feature
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

In [10]:
def supertrend(DF,n,m):
    """function to calculate Supertrend given historical candle data
        n = n day ATR - usually 7 day ATR is used
        m = multiplier - usually 2 or 3 is used"""
    df = DF.copy()
    df['ATR'] = atr(df,n)
    df["B-U"]=((df['high']+df['low'])/2) + m*df['ATR'] 
    df["B-L"]=((df['high']+df['low'])/2) - m*df['ATR']
    df["U-B"]=df["B-U"]
    df["L-B"]=df["B-L"]
    ind = df.index
    for i in range(n,len(df)):
        if df['close'][i-1]<=df['U-B'][i-1]:
            df.loc[ind[i],'U-B']=min(df['B-U'][i],df['U-B'][i-1])
        else:
            df.loc[ind[i],'U-B']=df['B-U'][i]    
    for i in range(n,len(df)):
        if df['close'][i-1]>=df['L-B'][i-1]:
            df.loc[ind[i],'L-B']=max(df['B-L'][i],df['L-B'][i-1])
        else:
            df.loc[ind[i],'L-B']=df['B-L'][i]  
    df['Strend']=np.nan
    for test in range(n,len(df)):
        if df['close'][test-1]<=df['U-B'][test-1] and df['close'][test]>df['U-B'][test]:
            df.loc[ind[test],'Strend']=df['L-B'][test]
            break
        if df['close'][test-1]>=df['L-B'][test-1] and df['close'][test]<df['L-B'][test]:
            df.loc[ind[test],'Strend']=df['U-B'][test]
            break
    for i in range(test+1,len(df)):
        if df['Strend'][i-1]==df['U-B'][i-1] and df['close'][i]<=df['U-B'][i]:
            df.loc[ind[i],'Strend']=df['U-B'][i]
        elif  df['Strend'][i-1]==df['U-B'][i-1] and df['close'][i]>=df['U-B'][i]:
            df.loc[ind[i],'Strend']=df['L-B'][i]
        elif df['Strend'][i-1]==df['L-B'][i-1] and df['close'][i]>=df['L-B'][i]:
            df.loc[ind[i],'Strend']=df['L-B'][i]
        elif df['Strend'][i-1]==df['L-B'][i-1] and df['close'][i]<=df['L-B'][i]:
            df.loc[ind[i],'Strend']=df['U-B'][i]
    return df['Strend']

In [11]:
def st_dir_refresh(ohlc,ticker):
    """function to check for supertrend reversal"""
    global st_dir
    if ohlc["st1"][-1] > ohlc["close"][-1] and ohlc["st1"][-2] < ohlc["close"][-2]:
        st_dir[ticker][0] = "red"
    if ohlc["st2"][-1] > ohlc["close"][-1] and ohlc["st2"][-2] < ohlc["close"][-2]:
        st_dir[ticker][1] = "red"
    if ohlc["st3"][-1] > ohlc["close"][-1] and ohlc["st3"][-2] < ohlc["close"][-2]:
        st_dir[ticker][2] = "red"
    if ohlc["st1"][-1] < ohlc["close"][-1] and ohlc["st1"][-2] > ohlc["close"][-2]:
        st_dir[ticker][0] = "green"
    if ohlc["st2"][-1] < ohlc["close"][-1] and ohlc["st2"][-2] > ohlc["close"][-2]:
        st_dir[ticker][1] = "green"
    if ohlc["st3"][-1] < ohlc["close"][-1] and ohlc["st3"][-2] > ohlc["close"][-2]:
        st_dir[ticker][2] = "green"

In [12]:
def sl_price(ohlc):
    """function to calculate stop loss based on supertrends"""
    st = ohlc.iloc[-1,[-3,-2,-1]]
    if st.min() > ohlc["close"][-1]:
        sl = (0.6*st.sort_values(ascending = True)[0]) + (0.4*st.sort_values(ascending = True)[1])
    if st.max() < ohlc["close"][-1]:
        sl = (0.6*st.sort_values(ascending = False)[0]) + (0.4*st.sort_values(ascending = False)[1])
    return round(sl,1)

In [13]:
def placeSLOrder(symbol,buy_sell,quantity,sl_price):    
    # Place an intraday stop loss order on NSE - handles market orders converted to limit orders
    if buy_sell == "buy":
        t_type=kite.TRANSACTION_TYPE_BUY
        t_type_sl=kite.TRANSACTION_TYPE_SELL
    elif buy_sell == "sell":
        t_type=kite.TRANSACTION_TYPE_SELL
        t_type_sl=kite.TRANSACTION_TYPE_BUY
    market_order = kite.place_order(tradingsymbol=symbol,
                    exchange=kite.EXCHANGE_NSE,
                    transaction_type=t_type,
                    quantity=quantity,
                    order_type=kite.ORDER_TYPE_MARKET,
                    product=kite.PRODUCT_MIS,
                    variety=kite.VARIETY_REGULAR)
    a = 0
    while a < 10:
        try:
            order_list = kite.orders()
            break
        except:
            print("can't get orders..retrying")
            a+=1
    for order in order_list:
        if order["order_id"]==market_order:
            if order["status"]=="COMPLETE":
                kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=t_type_sl,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                price=sl_price,
                                trigger_price = sl_price,
                                product=kite.PRODUCT_MIS,
                                variety=kite.VARIETY_REGULAR)
            else:
                kite.cancel_order(order_id=market_order,variety=kite.VARIETY_REGULAR)

In [14]:
def ModifyOrder(order_id,price):    
    # Modify order given order id
    kite.modify_order(order_id=order_id,
                    price=price,
                    trigger_price=price,
                    order_type=kite.ORDER_TYPE_SL,
                    variety=kite.VARIETY_REGULAR) 

In [15]:
def main(capital):
    a,b = 0,0
    while a < 10:
        try:
            pos_df = pd.DataFrame(kite.positions()["day"])
            break
        except:
            print("can't extract position data..retrying")
            a+=1
    while b < 10:
        try:
            ord_df = pd.DataFrame(kite.orders())
            break
        except:
            print("can't extract order data..retrying")
            b+=1
    
    for ticker in tickers:
        print("starting passthrough for - ",ticker)
        try:
            # Fetch 5 minute OHLC for the last 4 days
            ohlc = fetchOHLC(ticker,"5minute",4)
            ohlc["st1"] = supertrend(ohlc,7,3)
            ohlc["st2"] = supertrend(ohlc,10,3)
            ohlc["st3"] = supertrend(ohlc,11,2)
            st_dir_refresh(ohlc,ticker)
            # Quantity per ticker for 3000 rupees. HEnce 10 stocks would require at least 30000 rupees.
            quantity = int(capital/ohlc["close"][-1])
            if len(pos_df.columns)==0:
                if st_dir[ticker] == ["green","green","green"]:
                    placeSLOrder(ticker,"buy",quantity,sl_price(ohlc))
                if st_dir[ticker] == ["red","red","red"]:
                    placeSLOrder(ticker,"sell",quantity,sl_price(ohlc))
            if len(pos_df.columns)!=0 and ticker not in pos_df["tradingsymbol"].tolist():
                if st_dir[ticker] == ["green","green","green"]:
                    placeSLOrder(ticker,"buy",quantity,sl_price(ohlc))
                if st_dir[ticker] == ["red","red","red"]:
                    placeSLOrder(ticker,"sell",quantity,sl_price(ohlc))
            if len(pos_df.columns)!=0 and ticker in pos_df["tradingsymbol"].tolist():
                if pos_df[pos_df["tradingsymbol"]==ticker]["quantity"].values[0] == 0:
                    if st_dir[ticker] == ["green","green","green"]:
                        placeSLOrder(ticker,"buy",quantity,sl_price(ohlc))
                    if st_dir[ticker] == ["red","red","red"]:
                        placeSLOrder(ticker,"sell",quantity,sl_price(ohlc))
                if pos_df[pos_df["tradingsymbol"]==ticker]["quantity"].values[0] != 0:
                    order_id = ord_df.loc[(ord_df['tradingsymbol'] == ticker) & (ord_df['status'].isin(["TRIGGER PENDING","OPEN"]))]["order_id"].values[0]
                    ModifyOrder(order_id,sl_price(ohlc))
        except:
            print("API error for ticker :",ticker)

In [16]:
tickers = ["SUNPHARMA","CIPLA","NTPC","INFRATEL","INDUSINDBK","HEROMOTOCO","BAJFINANCE",
           "HCLTECH","DRREDDY","VEDL","SHREECEM","TITAN","TCS"] 
#tickers to track - recommended to use max movers from previous day
capital = 3000 #position size
st_dir = {} #directory to store super trend status for each ticker
for ticker in tickers:
    st_dir[ticker] = ["None","None","None"] 

In [ ]:
starttime=time.time()
timeout = time.time() + 60*60*6  # 60 seconds times 60 meaning 1 hrs
#This program only runs for the first 1 hour of trading
while time.time() <= timeout:
    try:
        main(capital)
        # Once the main program finishes executing and returns, We wait for 5 minutes. This strategy runs on 
        #5 minute trading candles
        time.sleep(300 - ((time.time() - starttime) % 300.0))
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit()        